In [ ]:
import copy
import glob
import os
import time
from collections import deque

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import algo
from arguments import get_args
from envs import make_vec_envs_ViZDoom
from model import Policy
from storage import RolloutStorage
from utils import get_vec_normalize
from visualize import visdom_plot

import os

In [ ]:
result_dir = "../vizdoom_record2/"
os.makedirs(result_dir, exist_ok=True)

reward_history = os.path.join(result_dir, "reward_history")
loss_history = os.path.join(result_dir, "loss_history")

#remove old record files
for f in [reward_history, loss_history]:
    try:
        os.remove(f)
    except OSError:
        pass

In [ ]:
args = get_args()

def ppo_hyper():
    args.algo = "ppo"
    args.use_gae = False
    args.tau = 0.95
    args.lr = 2.5e-4
    args.value_loss_coef = 1.0
    args.clip_param = 0.1
    args.num_processes = 8
    args.num_steps = 512
    args.num_mini_batch = 1
    args.gamma = 1.0
    args.entropy_coef = 0.0
    args.ppo_epoch = 1

def a2c_hyper():
    args.algo = "a2c"
    args.gamma = 1.0
    args.num_steps = 512
    args.num_processes = 8
    args.entropy_coef = 0.0

In [ ]:
a2c_hyper()
args

In [ ]:
num_updates = int(args.num_frames) // args.num_steps // args.num_processes
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [ ]:
torch.set_num_threads(1)
device = torch.device("cuda:0" if args.cuda else "cpu")
envs = make_vec_envs_ViZDoom(args.seed, args.num_processes, device)

actor_critic = Policy(envs.observation_space.shape, envs.action_space,
    base_kwargs={'recurrent': args.recurrent_policy})
actor_critic.to(device)

if args.algo == 'a2c':
    agent = algo.A2C_ACKTR(actor_critic, args.value_loss_coef,
                               args.entropy_coef, lr=args.lr,
                               eps=args.eps, alpha=args.alpha,
                               max_grad_norm=args.max_grad_norm)
else:
    agent = algo.PPO(actor_critic, args.clip_param, args.ppo_epoch, args.num_mini_batch,
                         args.value_loss_coef, args.entropy_coef, lr=args.lr,
                               eps=args.eps,
                               max_grad_norm=args.max_grad_norm)

rollouts = RolloutStorage(args.num_steps, args.num_processes,
                    envs.observation_space.shape, envs.action_space,
                    actor_critic.recurrent_hidden_state_size)

In [ ]:
obs = envs.reset()
rollouts.obs[0].copy_(obs)
rollouts.to(device)

recent_count = 50
episode_rewards = deque(maxlen=recent_count)
episode_lengths = deque(maxlen=recent_count)

In [ ]:
acc_step = []
acc_reward = []
acc_length = []

for j in range(num_updates):
    for step in range(args.num_steps):
        # Sample actions
        with torch.no_grad():
            value, action, action_log_prob, recurrent_hidden_states = actor_critic.act(
                    rollouts.obs[step],
                    rollouts.recurrent_hidden_states[step],
                    rollouts.masks[step])

        # Obser reward and next obs
        obs, reward, done, infos = envs.step(action)

        for info in infos:
            if 'Episode_Total_Reward' in info.keys():
                episode_rewards.append(info['Episode_Total_Reward'])
            if 'Episode_Total_Len' in info.keys():
                episode_lengths.append(info['Episode_Total_Len'])

        # If done then clean the history of observations.
        masks = torch.FloatTensor([[0.0] if done_ else [1.0]
                                   for done_ in done])
        rollouts.insert(obs, recurrent_hidden_states, action, action_log_prob, value, reward, masks)
        
    
    

    with torch.no_grad():
        next_value = actor_critic.get_value(rollouts.obs[-1],
                                            rollouts.recurrent_hidden_states[-1],
                                            rollouts.masks[-1]).detach()

    rollouts.compute_returns(next_value, args.use_gae, args.gamma, args.tau)

    value_loss, action_loss, dist_entropy = agent.update(rollouts)

    rollouts.after_update()
    
    total_num_steps = (j + 1) * args.num_processes * args.num_steps
    
    with open(loss_history, 'a') as the_file:
        the_file.write("{} {} {} {} \n".format(total_num_steps, value_loss, action_loss, dist_entropy))
    
    if len(episode_rewards) > 0:
        print("{} updates: avg reward = {}, avg length = {}".format(total_num_steps, np.mean(episode_rewards),
                                                               np.mean(episode_lengths)))
        
        with open(reward_history, 'a') as the_file:
            the_file.write('{} {} {} \n'.format(total_num_steps, np.mean(episode_rewards),
                                               np.mean(episode_lengths)))
        
    

In [ ]:
MODEL_SAVE_PATH = os.path.join(result_dir, "model.save")
torch.save(actor_critic.state_dict(), MODEL_SAVE_PATH)